<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Gather-Page-Rank-of-Non-Social-Media-External-Site" data-toc-modified-id="Gather-Page-Rank-of-Non-Social-Media-External-Site-1">Gather Page Rank of Non-Social Media External Site</a></span></li></ul></div>

In [3]:
import pickle

import features

import time
import json
import re

import requests
import json

api_key = '4gk44osw4cw84gcwkk8kkogw0okoc0g4o4c8kcww '

In [4]:
import tldextract
import pickle

In [5]:
json.dumps(['my_list','google.com'])

'["my_list", "google.com"]'

In [11]:
# query = {'domains':{0:'facebook.com'},'API-OPR':api_key}

query = {
    'domains[]': ['thomasmkavanagh.com'],
    'API-OPR':api_key
}

In [12]:
pr = requests.get(url='https://openpagerank.com/api/v1.0/getPageRank',params=query)

In [13]:
pr.status_code

200

In [14]:
pr.text

'{"status_code":200,"response":[{"status_code":200,"error":"","page_rank_integer":0,"page_rank_decimal":0.26,"rank":"153096046","domain":"thomasmkavanagh.com"}],"last_updated":"16th Aug 2019"}'

In [ ]:
pr = requests.get(url='https://openpagerank.com/api/v1.0/getPageRank?API-OPR=4gk44osw4cw84gcwkk8kkogw0okoc0g4o4c8kcww&domains[]=google.com')

In [ ]:

pr = requests.get(url='https://openpagerank.com/api/v1.0/getPageRank?API-OPR=4gk44osw4cw84gcwkk8kkogw0okoc0g4o4c8kcww&domains[]=http://www.winemag.com')

In [ ]:
pr.status_code

In [ ]:
pr.text

In [ ]:
json.loads(pr.text)['response'][0]['page_rank_decimal']

## Gather Page Rank of Non-Social Media External Site

Podcasts can optionally include one external site for their channel that isn't Facebook, Twitter, Instagram, Youtube, etc.

We can grab the Google PageRank of this external site, if it exists, by sending a GET request to the Open Page Rank API. This can give us a measure of the reputability and relative exposure of a podcasts

The Open Page Rank all of the Google PageRank 

We will unload (any) version of the sanitized dataframe, in order to gather a list of channels in order to create a dictionary.

In [ ]:
df_version = 'merged_10-08_13-54.pickle'
import_path = '../scraped/merged/' + df_version
with open(import_path, 'rb') as file:
    raw_df = pickle.load(file)

In [ ]:
raw_df['ch_feed-socials'].sample(10).values

In [ ]:
sample = raw_df['ch_feed-socials'].sample(10).values[0]
sample

In [ ]:
for link in sample:
    social_domains = ['twitter', 'facebook', 'youtube', 'instagram']
    for domain in social_domains:
        pattern = '.*' + domain + '.*'
#         print(pattern, link)
#         print(re.match(pattern, link))
        if re.match(pattern, link):
            break
        else:
            print(link)
            break

In [ ]:
ch_feed = ['https://twitter.com/bulletproofexec',
 'https://www.facebook.com/bulletproofexecutive',
 'https://blog.daveasprey.com/category/podcasts/']

for link in ch_feed:
    external = link
    print('testing ', link)
    social_domains = ['twitter', 'facebook', 'youtube', 'instagram']
    for domain in social_domains:
        pattern = '.*' + domain + '.*'
#         print(pattern, link)
#         print(re.match(pattern, link))
        if re.match(pattern, link):
            print('matches ',domain)
            external = False

print('match: ', external)

In [ ]:
def return_external_site_domain(ch_feed_socials):
    '''
    
    Return a non-social external domain from a list of domains.abs
    
    If one does not exist, return False
    
    
    '''
    
#     print(ch_feed_socials)
    
    if not ch_feed_socials:
        return False
    
    for link in ch_feed_socials:
        external = link
#         print('testing ', link)
        social_domains = ['twitter', 'facebook', 'youtube', 'instagram']
        for domain in social_domains:
            pattern = '.*' + domain + '.*'
    #         print(pattern, link)
    #         print(re.match(pattern, link))
            if re.match(pattern, link):
#                 print('matches ',domain)
                external = False

    return external

In [ ]:
raw_df['external_site'] = raw_df['ch_feed-socials'].apply(return_external_site_domain)

In [ ]:
def get_page_rank(domain_name):
    
    print('getting OPR for: ',domain_name)
    
#     print(domain_name)
    
    query = {
        'domains[]': domain_name,
        'API-OPR':api_key
    }
    
    response = json.loads(
        requests.get(url='https://openpagerank.com/api/v1.0/getPageRank',params=query).text
    )['response'][0]
    
#     print(response)
    
    page_rank_dec = response['page_rank_decimal']
    
#     print(page_rank_dec)
    
    if page_rank_dec == '':
        print('no rank')
        return 0
    else:
        print(page_rank_dec)
        return page_rank_dec

In [ ]:
def extract_domain(site):
    try:
        domain_name = tldextract.extract(site).domain + '.' + tldextract.extract(site).suffix
        return domain_name
    except:
        return ''

In [ ]:
raw_df['domain'] = raw_df.external_site.apply(extract_domain)

In [ ]:
raw_df.shape

In [ ]:
raw_df.domain.nunique()

In [ ]:
domains_list = list(raw_df.domain.unique())

In [ ]:
domains_list_sample = domains_list[:100]
domains_list_sample[:5]

In [ ]:
# test on batch domains
# query = {'domains':{0:'facebook.com'},'API-OPR':api_key}

query = {
    'domains[]': domains_list_sample,
    'API-OPR':api_key
}

pr = requests.get(url='https://openpagerank.com/api/v1.0/getPageRank',params=query)

r = json.loads(pr.text)['response']

In [ ]:
r[:5]

In [ ]:
def get_page_rank_batch(domains):
    
    print(f'getting {len(domains)} domains via OPR: ', domains[0], '....', domains[-1])
    
    query = {
        'domains[]': domains,
        'API-OPR':api_key
    }

    pr = requests.get(url='https://openpagerank.com/api/v1.0/getPageRank',params=query)

    multi_response = json.loads(pr.text)['response']
    
    return multi_response

In [ ]:
len(domains_list)

In [ ]:
for i in range(0,len(domains_list)//100 + 1):
    domain_list_batch = domains_list[i*100:i*100+100]
    print(domain_list_batch)

In [ ]:
all_domains_dict = {}
for i in range(0, len(domains_list), 99):
    domain_list_batch = domains_list[i:i + 99]
#     print(domain_list_batch)
    response_batch = get_page_rank_batch(domain_list_batch)
    batch_dict = {}
    for r in response_batch:
        domain = r['domain']
        prd = r['page_rank_decimal']
        rank = r['rank']
        batch_dict[domain] = {'page_rank_dict': prd, 'rank': rank}
    all_domains_dict.update(batch_dict)

all_domains_dict

In [ ]:
with open('../social_metrics/external_domains/external_domain_dicts.pickle','wb') as file:
    pickle.dump(all_domains_dict,file)

In [2]:
with open('../social_metrics/external_domains/external_domain_dicts.pickle','rb') as file:
    domains = pickle.load(file)
    
domains.keys()

dict_keys(['tablegroup.com', 'learningleadershow.com', 'johnmaxwellteam.com', 'spreaker.com', 'rework.fm', 'askamanager.org', 'kitces.com', 'doseofleadership.com', 'labcoatagents.com', 'ecommercefuel.com', 'katenorthrup.com', '15five.com', 'leecockerell.com', 'eatsleepworkrepeat.com', 'podbean.com', 'readtoleadpodcast.com', 'indiehackers.com', 'lawyerist.com', 'wmleadership.com', 'strategicpodcasts.com', 'libsyn.com', 'aba.com', 'villageglobal.vc', 'fitnessbusinesspodcast.com', 'pmmasterprep.com', 'becomingyourbest.com', 'thesaleswhisperer.com', 'tammylenski.com', 'laurieruettimann.com', 'l3leadership.org', 'drdannymatta.com', 'letgoandlead.com', 'privatepracticeworkshop.com', 'btgvoice.com', 'krisplachy.com', 'killerinnovations.com', 'nationalpropertymanagement.com', 'jasonswenk.com', 'podomatic.com', 'photobizx.com', 'businessbootcamppodcast.com', 'hubspot.com', 'whooshkaa.com', 'cuinsight.com', 'legaltalknetwork.com', 'paolaelizaga.com', 'harmonbrothers.com', 'myhrfuture.com', 'rock